<a href="https://colab.research.google.com/github/franov/analytics/blob/main/RL_Iteraci%C3%B3n_de_valor_y_de_pol%C3%ADtica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Resolviendo MDP

This notebook covers two fundamental algorithms to solve MDPs namely **Value Iteration** and **Policy Iteration**.

## Markov Decision Process - MDP

In MDP, there is an agent. The agent choose an action $a_{t}$ at time $t$ and as a consequance the enviorment changes.
Here The evniorment is world around the agent. After the action the enviorment state changes to $s_{t+1}$.
A reward might be emitted assciated with what just happened and then this process repeats. ![](nb_images/mdp.png)

So, there is a feedback cycle in that the next action you take, the next decision you make is in a situation that's the consiquence of what you did before.

## 1. Importando bibliotecas necesarias

In [ ]:
!sudo apt install python3.9
!pip install numpy==2.0

In [ ]:
import numpy as np
np.bool8 = np.bool
import gym
import gym.spaces as spaces
import time

In [ ]:
# action mapping for display the final result
action_mapping = {
    3: '\u2191', # UP
    2: '\u2192', # RIGHT
    1: '\u2193', # DOWN
    0: '\u2190' # LEFT
}
print(' '.join([action_mapping[i] for i in range(4)]))

← ↓ → ↑


## 2. Preparar el ambiente de Gym para las interacciones



```
# This is formatted as code
```
Se define una función que toma un ambiente de Gym y ejecuta un número de interacciones de acuerdo a una política dada
.

In [ ]:
def play_episodes(enviorment, n_episodes, policy, random = False):
    """
    Esta función juega una cantidad de episodios especificados en n_episodes sigueindo una política dada, o tomando acciones al azar desde action_space

    Parámetros:
        enviorment: ambiente de Gym
        n_episodes: número de episodios a correr
        policy: Política a seguir en cada episodio.
        random: Flag para tomar acciones aleatorias. Si es True, no debería seguirse una política y la acción sería aleatoria

    Retorno:
        wins: Número total de wins jugando n_episodios
        total_reward: Recompensa total de los n_episodes
        avg_reward: Recompensa promedio de los n_episodes

    """
    # intialize wins and total reward
    wins = 0
    total_reward = 0

    # loop over number of episodes to play
    for episode in range(n_episodes):

        # flag to check if the game is finished
        terminated = False

        # reset the enviorment every time when playing a new episode
        state = enviorment.reset()

        while not terminated:

            # check if the random flag is not true then follow the given policy other wise take random action
            if random:
                action = enviorment.action_space.sample()
            else:
                action = policy[state]



            # take the next step
            next_state, reward,  terminated, info = enviorment.step(action)



            enviorment.render()



            # accumalate total reward
            total_reward += reward

            # change the state
            state = next_state

            # if game is over with positive reward then add 1.0 in wins
            if terminated and reward == 1.0:
                wins += 1

    # calculate average reward
    average_reward = total_reward / n_episodes

    return wins, total_reward, average_reward


## 3. Solve for Value Iteration.

![](nb_images/value_iter.png)

In [ ]:

def one_step_lookahead(env, state, V , discount_factor = 0.99):
    """
    Calcula la función de valor del estado

    Argumentos:
        env: ambiente de gym
        state: Estado actual
        V: Valor estimado para cada estado, vector de tamaño observation_space.n
        discount_factor:factor de descuento

    Retorno:
        action_values: Valor esperado de cada acción en un estado, vector de tamaño action_space.n
    """

    # inicializando vector de acciones de valor
    action_values = np.zeros(env.action_space.n)

    # loop sobre las acciones que pueden tomarse en un ambiente
    for action in range(env.action_space.n):
        # loop sobre la distribución P_sa .
        for probablity, next_state, reward, info in env.P[state][action]:
             #si se está en un estado s y toma una acción a, se suma sobre todos los posibles estados a los que se puede llegar.
            action_values[action] += probablity * (reward + (discount_factor * V[next_state]))

    return action_values

In [ ]:
def update_policy(env, policy, V, discount_factor):

    """
    Función para actualizar una política en base a una función de valor pasada como input.

    Argumentos:
        env: ambiente de Gym.
        policy: política a actualizar
        V: Valor estimado para cada estado, vector de tamaño observation_space.n
        discount_factor: factor de descuento
    Retorno:
        policy: política actualizada en base a la función de valor-estado 'V'
    """

    for state in range(env.observation_space.n):
        # calcular el valor de estado-acción para cada estado
        action_values = one_step_lookahead(env, state, V, discount_factor)

        # choose the action which maximizez the state-action value.
        policy[state] =  np.argmax(action_values)

    return policy


In [ ]:
def value_iteration(env, discount_factor = 0.999, max_iteration = 1000):
    """
    Algoritmo para resolver un MPD.

    Argumentos:
        env: ambiente Gym.
        discount_factor: factor de descuento
        max_iteration:máximo número de iteraciones a ejecutar.

    Retorno:
        V: Función de valor de estado óptima. Vector de tamaño observacion_space.n
        optimal_policy: Optimal policy. Vector of length observacion_space.n.

    """
    # intialize value fucntion
    V = np.zeros(env.observation_space.n)

    # iterate over max_iterations
    for i in range(max_iteration):

        #  keep track of change with previous value function
        prev_v = np.copy(V)

        # loop over all states
        for state in range(env.observation_space.n):

            # Asynchronously update the state-action value
            #action_values = one_step_lookahead(env, state, V, discount_factor)

            # Synchronously update the state-action value
            action_values = one_step_lookahead(env, state, prev_v, discount_factor)

            # select best action to perform based on highest state-action value
            best_action_value = np.max(action_values)

            # update the current state-value fucntion
            V[state] =  best_action_value

        # if policy not changed over 10 iterations it converged.
        if i % 10 == 0:
            # if values of 'V' not changing after one iteration
            if (np.all(np.isclose(V, prev_v))):
                print('Value converged at iteration %d' %(i+1))
                break

    # intialize optimal policy
    optimal_policy = np.zeros(env.observation_space.n, dtype = 'int8')

    # update the optimal polciy according to optimal value function 'V'
    optimal_policy = update_policy(env, optimal_policy, V, discount_factor)

    return V, optimal_policy

## Evaluando el Algoritmo

In [ ]:
enviorment = gym.make('FrozenLake-v1')
tic = time.time()
opt_V, opt_Policy = value_iteration(enviorment.env, max_iteration = 1000)
toc = time.time()
elapsed_time = (toc - tic) * 1000
print (f"Time to converge: {elapsed_time: 0.3} ms")
print('Optimal Value function: ')
print(opt_V.reshape((4, 4)))
print('Final Policy: ')
print(opt_Policy)
print(' '.join([action_mapping[int(action)] for action in opt_Policy]))

In [ ]:
n_episode = 10
wins, total_reward, avg_reward = play_episodes(enviorment, n_episode, opt_Policy, random = False)

In [ ]:
print(f'Total wins with value iteration: {wins}')
print(f"Average rewards with value iteration: {avg_reward}")

## 4. Solve for Policy Iteration

![](nb_images/policy_iter.png)

In [ ]:
def policy_eval(env, policy, V, discount_factor):
   # COMPLETAR

    return policy_value

In [ ]:
def policy_iteration(env, discount_factor = 0.999, max_iteration = 1000):
    # COMPLETAR
    return V, policy

## Test Policy Iteration

In [ ]:
enviorment2 = gym.make('FrozenLake-v1')
tic = time.time()
opt_V2, opt_policy2 = policy_iteration(enviorment2.env, discount_factor = 0.999, max_iteration = 10000)
toc = time.time()
elapsed_time = (toc - tic) * 1000
print (f"Time to converge: {elapsed_time: 0.3} ms")
print('Optimal Value function: ')
print(opt_V2.reshape((4, 4)))
print('Final Policy: ')
print(opt_policy2)
print(' '.join([action_mapping[(action)] for action in opt_policy2]))

In [ ]:
n_episode = 10
wins, total_reward, avg_reward = play_episodes(enviorment2, n_episode, opt_policy2, random = False)

In [ ]:
print(f'Total wins con iteración de política: {wins}')
print(f"Recompensas promedio con iteración de política: {avg_reward}")

## EJERCICIOS:
1.- Repase los algoritmos de iteración de valor y de política de la clase pasada

2.- Revise, comprenda y ejecute el código actual: ¿cuántas iteraciones tarda en converger el método de iteración de valor?

3.- Complete el código de iteración de política en la sección respectiva: ¿cuántas iteraciones tarda en converger este método?

4.- Comparando ambos métodos:  ¿cuál converge más rápido? puede evaluar el uso de recursos de cómputo?

5.- Adapte este código para que se ejecute sobre otro ambiente de Gym (por ej cliff walking) y responda las preguntas de 2 a 4 para ese ambiente